# Import Library

In [14]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

Only DS

In [15]:
cwd = os.getcwd()
# 폴더 위치 변경
os.chdir("C:\\Users\hyunj\\Dropbox\\종합설계프로젝트\\중요\\데이터")

data = pd.read_csv('LD.csv', encoding='cp949')
print('LD의 Feature : ' , data.columns)
data.head()

LD의 Feature :  Index(['Case ID', 'Activity', 'Start Timestamp', 'Complete Timestamp',
       'Variant', 'Variant index', 'VESSEL', 'MCHN_ID', 'FULL_EMPTY', 'POD',
       'BLOCK_BAY'],
      dtype='object')


,Case ID,Activity,Start Timestamp,Complete Timestamp,Variant,Variant index,VESSEL,MCHN_ID,FULL_EMPTY,POD,BLOCK_BAY
0,CLHU2541691-2,LD-YARDSIDE,2018-01-31 22:40:52.000,2018-01-31 22:43:25.000,Variant 1,1,ALAP-001/2018,TC223,F,CAVAN,2B-13
1,CLHU2541691-2,LD-MOVE,2018-01-31 22:43:25.000,2018-01-31 22:55:23.000,Variant 1,1,ALAP-001/2018,YT536,F,CAVAN,2B-13
2,CLHU2541691-2,LD-QUAYSIDE,2018-01-31 22:55:23.000,2018-01-31 22:57:56.000,Variant 1,1,ALAP-001/2018,GC110,F,CAVAN,2B-13
3,CNIU1183311-1,LD-YARDSIDE,2018-01-31 22:26:15.000,2018-01-31 22:28:48.000,Variant 1,1,ALAP-001/2018,TC221,F,CAVAN,1B-25
4,CNIU1183311-1,LD-MOVE,2018-01-31 22:28:48.000,2018-01-31 22:29:09.000,Variant 1,1,ALAP-001/2018,YT525,F,CAVAN,1B-25


In [16]:
# 필요없는 열 삭제 & Feature name 변경
data.drop('Variant', axis = 1, inplace = True)
data.drop('Variant index', axis = 1, inplace = True)

data.rename(columns = {'Case ID':'CONTAINER_NO',
                       'Activity':'JOB_TYPE',
                       'Start Timestamp': 'JOB_START_DT',
                       'Complete Timestamp' : 'JOB_END_DT'},
                        inplace=True)
print('LD의 Feature : ' , data.columns)
print('LD의 SHAPE : ', data.shape)
data.head()

LD의 Feature :  Index(['CONTAINER_NO', 'JOB_TYPE', 'JOB_START_DT', 'JOB_END_DT', 'VESSEL',
       'MCHN_ID', 'FULL_EMPTY', 'POD', 'BLOCK_BAY'],
      dtype='object')
LD의 SHAPE :  (125646, 9)


,CONTAINER_NO,JOB_TYPE,JOB_START_DT,JOB_END_DT,VESSEL,MCHN_ID,FULL_EMPTY,POD,BLOCK_BAY
0,CLHU2541691-2,LD-YARDSIDE,2018-01-31 22:40:52.000,2018-01-31 22:43:25.000,ALAP-001/2018,TC223,F,CAVAN,2B-13
1,CLHU2541691-2,LD-MOVE,2018-01-31 22:43:25.000,2018-01-31 22:55:23.000,ALAP-001/2018,YT536,F,CAVAN,2B-13
2,CLHU2541691-2,LD-QUAYSIDE,2018-01-31 22:55:23.000,2018-01-31 22:57:56.000,ALAP-001/2018,GC110,F,CAVAN,2B-13
3,CNIU1183311-1,LD-YARDSIDE,2018-01-31 22:26:15.000,2018-01-31 22:28:48.000,ALAP-001/2018,TC221,F,CAVAN,1B-25
4,CNIU1183311-1,LD-MOVE,2018-01-31 22:28:48.000,2018-01-31 22:29:09.000,ALAP-001/2018,YT525,F,CAVAN,1B-25


In [17]:
# 통합할 데이터 호출
che_data = pd.read_csv('CHE I perspective.csv')
gate_data = pd.read_csv('GI perspective.csv')
qc_data = pd.read_csv('QC perspective.csv')

gate_data.drop('Variant', axis = 1, inplace = True)
gate_data.drop('Variant index', axis = 1, inplace = True)

gate_data.rename(columns = {'Case ID':'CONTAINER_NO',
                       'Activity':'JOB_TYPE',
                       'Start Timestamp': 'JOB_START_DT',
                       'Complete Timestamp' : 'JOB_END_DT'},
                        inplace=True)

In [18]:
# 최종 DS csv 파일 - 자세한 것은 파워포인트 참고
temp_df = pd.DataFrame(columns=["CONTAINER_NO", "VESSEL", "MCHN_ID", "FULL_EMPTY", "JOB_TYPE", "POD", "BLOCK_BAY", "JOB_START_DT", "JOB_END_DT",
                                "CHE_ID", "ITV_ID", "QC_ID", "OUTSERVICE", "QUEUE_POSITION", "H_BOX",
                                "OUTVESSEL", "COUNTRY_NEXTPOD", "PORT_NEXTPOD", "TML_OUT_DT"])
print('최종적으로 사용할 데이터 셋의 Column 수는' , len(temp_df.columns))

# temp에서 추가하고 이를 last에서 concat하는 걸로 - 왜냐하면 temp에서 containerid별로 가져올 것이기 때문에, 이렇게 해주는 이유는 정렬하기 위해서 
# (사실 다 추가하고 정렬하면 괜찮지만, 작업 중간 중간에 볼 것이기 때문에 이렇게 함 - 메모리 이슈는 고려하지 않음) 
last_df = temp_df.copy()

최종적으로 사용할 데이터 셋의 Column 수는 19


In [19]:
unique_container_list = list(data['CONTAINER_NO'].unique())
print(len(unique_container_list))

41882


In [20]:
csv_list = [gate_data, che_data, qc_data]

for csv in csv_list :
    temp = max(csv['CONTAINER_NO'].value_counts())
    print(temp)

1
3
2


In [21]:
only_gate_data = gate_data[gate_data['CONTAINER_NO'].isin(unique_container_list)]
print(len(only_gate_data['CONTAINER_NO'].unique()))
value_counts_gate = only_gate_data['CONTAINER_NO'].value_counts()

36941


In [22]:
only_che_data = che_data[che_data['CONTAINER_NO'].isin(unique_container_list)]
print(len(only_che_data['CONTAINER_NO'].unique()))
value_counts_che = only_che_data['CONTAINER_NO'].value_counts()

41882


In [23]:
# GI에 대한 정보가 있는 것들
len(value_counts_che[value_counts_che==2])

36920

In [24]:
only_qc_data = qc_data[qc_data['CONTAINER_NO'].isin(unique_container_list)]
print(len(only_qc_data['CONTAINER_NO'].unique()))
value_counts_qc = only_qc_data['CONTAINER_NO'].value_counts()

41882


유일하게 che만 2개의 값이 존재할 수 있는 것을 확인 - 하나인 경우는 GI에 대한 정보가 누락된 것임

In [25]:
# 데이터 우선 통합 - Container perspective 기준!
temp_df = pd.concat([temp_df,data])

In [26]:
# perspetive를 비교하면서 매치되지 않는 CONTAINER_NO 개수를 기록하기 위한 변수 선
for container in tqdm(unique_container_list, desc = "데이터 통합 중") : 
    criteria_df = temp_df[temp_df['CONTAINER_NO'] == container].reset_index(drop=True)
    #! Gate perspective에서 정보 추출 - GO & GI (DS에선 GO!)
    gate_temp_df = gate_data[gate_data['CONTAINER_NO'] == container].reset_index(drop=True)
    
    # container perspective에 있는 CONTAINER_NO가 gate_perspective에 없는 경우도 고려해보아야 함
    if len(gate_temp_df) != 0 :
        # gate_temp_df의 Gate Out에 대한 정보를 temp_df에 전달
        criteria_df = pd.concat([criteria_df, gate_temp_df])
        criteria_df = criteria_df.reset_index(drop=True)
    
    #! CHE I perspective에서 정보 추출 - MOVE
    #* CHE I perspective에는 GI, MOVE의 정보가 있음
    che_temp_df = che_data[che_data['CONTAINER_NO'] == container].reset_index(drop=True)
    
    gi_che_temp_df = che_temp_df[che_temp_df['JOB_TYPE'] == "GI"].reset_index(drop=True)
    move_che_temp_df = che_temp_df[che_temp_df['JOB_TYPE'] == "LD"].reset_index(drop=True)
    # MOVE와 연관이 있는 것을 확인하였지만, 혹시 모르니 두 개의 ID를 비교하여 정보 추가
    # 시간으로 하니 정확한 비교가 안됨
    if(len(che_temp_df) != 0) :
        # move에 관한 정보가 있으면
        if (len(move_che_temp_df) != 0) :
            if((criteria_df.loc[2, 'MCHN_ID'] == move_che_temp_df.loc[0, 'QC_ID'])
            & (criteria_df.loc[1, 'MCHN_ID'] == move_che_temp_df.loc[0, 'ITV_ID'])) :
                che_id = move_che_temp_df.loc[0, 'MCHN_ID']
                itv_id = move_che_temp_df.loc[0, 'ITV_ID']
                qc_id = move_che_temp_df.loc[0, 'QC_ID']
                criteria_df.loc[1, 'CHE_ID'] = che_id
                criteria_df.loc[1, 'ITV_ID'] = itv_id
                criteria_df.loc[1, 'QC_ID'] = qc_id
                
        # Gate in에 관한 정보가 있으면
        if (len(gi_che_temp_df) != 0) :
            che_id = gi_che_temp_df.loc[0, 'MCHN_ID']
            itv_id = gi_che_temp_df.loc[0, 'ITV_ID']
            criteria_df.loc[3, 'CHE_ID'] = che_id
            criteria_df.loc[3, 'ITV_ID'] = itv_id
    
    #! QC perspective에서 정보 추출 - QUAYSIDE
    qc_temp_df = qc_data[qc_data['CONTAINER_NO'] == container].reset_index(drop=True)
    
    # QUAYSIDE와 연관이 있는 것을 확인하였지만, 혹시 모르니 두 개의 perspective의 ID 비교하기
    crane_check = qc_temp_df.loc[0, 'QRANE_NO'] == criteria_df.loc[2, 'MCHN_ID']
    mchn_check = qc_temp_df.loc[0, 'MCHN_ID'] == criteria_df.loc[1, 'MCHN_ID']
    if (len(qc_temp_df) != 0) :
        queue_position = qc_temp_df.loc[0, 'QUEUE_POSITION']
        h_box = qc_temp_df.loc[0, 'H_BOX']
        criteria_df.loc[2, 'QUEUE_POSITION'] = queue_position
        criteria_df.loc[2, 'H_BOX'] = h_box
    
    #! Vessel perspective은 LD에서 통합할 필요가 없음
    
    #! 각 정보를 전달한 뒤에 last_df에 concat
    last_df = pd.concat([criteria_df, last_df])
    last_df = last_df.reset_index(drop=True)
    

데이터 통합 중: 100%|██████████| 41882/41882 [44:42<00:00, 15.61it/s] 


In [27]:
last_df.to_csv('Integrated LD.csv', encoding='cp949', index=False)